<a href="https://colab.research.google.com/github/msommerfe/CAS-Applied-Data-Science/blob/master/Module6_Deep_Learning/Object_DetectionV8_GoogleColab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import ssl
import urllib
import os
import time

In [3]:
pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 719.5/719.5 kB 9.8 MB/s eta 0:00:00


In [5]:
from ultralytics import YOLO

In [6]:
!nvidia-smi

Wed Mar  6 10:06:44 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [7]:
#!python train.py --img 1280 --batch 16 --epochs 500 --data /content/drive/MyDrive/Colab_Notebooks/Data/Try_EVN_1200x900/YoloTrainingPath.yaml --weights yolov5s.pt
yamlPath = '/content/drive/MyDrive/Colab_Notebooks/Data/Try_EVN_1280x960/YoloTrainingPath.yaml'
model = YOLO('yolov8n.pt')

results = model.train(data = yamlPath, epochs = 80, imgsz=320)

100%|██████████| 6.23M/6.23M [00:00<00:00, 96.5MB/s]


Ultralytics YOLOv8.1.24 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/drive/MyDrive/Colab_Notebooks/Data/Try_EVN_1280x960/YoloTrainingPath.yaml, epochs=80, time=None, patience=100, batch=16, imgsz=320, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, 

100%|██████████| 755k/755k [00:00<00:00, 23.5MB/s]


Overriding model.yaml nc=80 with nc=18

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytic

train: Scanning /content/drive/MyDrive/Colab_Notebooks/Data/Try_EVN_1280x960/train/labels.cache... 635 images, 4 backgrounds, 0 corrupt: 100%|██████████| 639/639 [00:00<?, ?it/s]


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


val: Scanning /content/drive/MyDrive/Colab_Notebooks/Data/Try_EVN_1280x960/validate/labels.cache... 22 images, 2 backgrounds, 0 corrupt: 100%|██████████| 24/24 [00:00<?, ?it/s]


Plotting labels to runs/detect/train/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000455, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 320 train, 320 val
Using 2 dataloader workers
Logging results to runs/detect/train
Starting training for 80 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/80     0.826G      3.441      5.983      2.578         29        320:  25%|██▌       | 10/40 [00:20<01:01,  2.04s/it]


KeyboardInterrupt: 

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"
!cp -av "/content/runs" "/content/drive/MyDrive/Colab_Notebooks/Data/Try_EVN_1200x900"

'/content/runs/detect/train/weights/last.pt' -> '/content/drive/MyDrive/Colab_Notebooks/Data/Try_EVN_1200x900/runs/detect/train/weights/last.pt'
'/content/runs/detect/train/weights/best.pt' -> '/content/drive/MyDrive/Colab_Notebooks/Data/Try_EVN_1200x900/runs/detect/train/weights/best.pt'
'/content/runs/detect/train/args.yaml' -> '/content/drive/MyDrive/Colab_Notebooks/Data/Try_EVN_1200x900/runs/detect/train/args.yaml'
'/content/runs/detect/train/events.out.tfevents.1709457712.8655d8aec19e.997.0' -> '/content/drive/MyDrive/Colab_Notebooks/Data/Try_EVN_1200x900/runs/detect/train/events.out.tfevents.1709457712.8655d8aec19e.997.0'
'/content/runs/detect/train/labels_correlogram.jpg' -> '/content/drive/MyDrive/Colab_Notebooks/Data/Try_EVN_1200x900/runs/detect/train/labels_correlogram.jpg'
'/content/runs/detect/train/labels.jpg' -> '/content/drive/MyDrive/Colab_Notebooks/Data/Try_EVN_1200x900/runs/detect/train/labels.jpg'
'/content/runs/detect/train/train_batch0.jpg' -> '/content/drive/MyDri

# Neuer Abschnitt